In [ ]:
import requests as req
import ujson as js
def getPoloniexHistorical(symbol='BTC_XMR', period=14400):
    url = 'https://poloniex.com/public?command=returnChartData&currencyPair=%s&start=1405699200&end=9999999999&period=%s' % (symbol, period)
    res = req.get(url)
    #res.text
    li = js.loads(res.text)
    print li
    df = p.DataFrame(li)
    with p.option_context('display.max_rows', 4000, 'display.max_columns', 4000, 'display.width', 1000000):
        print df.tail(5)
    return df
#df = getPoloniexHistorical('BTC_XMR')

In [ ]:
#$x('//table[@id="marketBTC"]//td[2]/text()')
btc = ["XRP", "STR", "ETH", "LTC", "ETC", "XMR", "DGB", "DASH", "FCT", "DOGE", "BTS", "XEM", "GNO", "SC", "GNT", "ZEC", "STEEM", "MAID", "PASC", "SYS", "LSK", "CLAM", "STRAT", "DCR", "XCP", "REP", "NXT", "POT", "FLDC", "VTC", "NAV", "PINK", "ARDR", "GAME", "BCN", "BURST", "VRC", "BELA", "AMP", "SJCX", "LBC", "XBC", "PPC", "XVC", "GRC", "NAUT", "BTM", "OMNI", "BCY", "EXP", "EMC2", "SBD", "NOTE", "HUC", "VIA", "BLK", "NMC", "XPM", "RIC", "NXC", "RADS", "NEOS", "FLO", "BTCD"]
#$x('//table[@id="marketETH"]//td[2]/text()')
eth = ["GNO", "ETC", "GNT", "ZEC", "REP", "STEEM", "LSK"]
#$x('//table[@id="marketXMR"]//td[2]/text()')
xmr = ["LTC", "ZEC", "DASH", "NXT", "MAID", "BCN", "BTCD", "BLK"]
#$x('//table[@id="marketUSDT"]//td[2]/text()')
usdt = ["BTC", "XRP", "STR", "LTC", "ETH", "ETC", "XMR", "DASH", "ZEC", "NXT", "REP"]

In [ ]:
from matplotlib import pyplot as plt
from pylab import rcParams
import seaborn as sns
sns.set()
%pylab inline
rcParams['figure.figsize'] = 30, 5
df = getPoloniexHistorical(symbol='BTC_DOGE', period=300)
df.ix[:, 'open high low close'.split(' ')].plot()
plt.show()

In [ ]:
btc = 'DASH ETH FCT GNO LTC XMR REP XRP ZEC'.split(' ')
periods = '1 5 15 30 60 240 14400'.split(' ')
periods = [300, 900, 1800, 7200, 14400, 86400]
#periods = [1, 5, 15, 30, 60, 3600, 14400, 86400]
print periods
from matplotlib import pyplot as plt
from pylab import rcParams
import seaborn as sns
sns.set()
%pylab inline
rcParams['figure.figsize'] = 30, 5
for i in btc:
    print i
    df = getPoloniexHistorical(symbol='BTC_%s' % i, period=1)
    df.ix[:, 'open high low close'.split(' ')].plot()
    plt.show()
    print '====='

In [ ]:
# https://www.google.com.ar/search?q=coinmarketcap+ohlc&oq=coinmarketcap+ohlc&aqs=chrome..69i57.3815j0j7&client=ubuntu&sourceid=chrome&ie=UTF-8
#https://poloniex.com/public?command=returnChartData&currencyPair=BTC_XMR&start=1405699200&end=9999999999&period=14400
# https://poloniex.com/public?command=returnChartData&currencyPair=BTC_XMR&start=1405699200&end=9999999999&period=14400
    
# poloniex
import drest
api = drest.API('https://poloniex.com/')
response = api.make_request('GET', '/public?command=returnChartData&currencyPair=BTC_XMR&start=1405699200&end=9999999999&period=14400')
print response.data

#df = p.DataFrame(response.data)#.transpose()
#pf(df)
#df

In [ ]:
import pandas as p
from qoreliquid import pf

# bitmex
import drest
#api = drest.API('http://socket.coincap.io/')
api = drest.API('https://www.bitmex.com/api/v1')
#response = api.make_request('GET', '/trade?count=100&reverse=false')
#response = api.make_request('GET', '/instrument')
response = api.make_request('GET', '/instrument/indices')
#print response.data

df = p.DataFrame(response.data)#.transpose()
#pf(df)
df